In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
url = "https://www.wien.gv.at/statistik/verkehr-wohnen/tabellen/kfz-bestand-bez.html"
resp = requests.get(url)
resp.raise_for_status()  # Fehlermeldung, falls die Seite nicht erreichbar ist

In [19]:
soup = BeautifulSoup(resp.content, "html.parser")
table = soup.find("table", class_="BDE-table-frame")

In [20]:
rows = table.find_all("tr")[2:]

In [21]:
data = []
for tr in rows:
    cells = tr.find_all("td")
    if len(cells) != 8:
        continue
    row = []
    for td in cells:
        txt = td.get_text(strip=True)
        # Tausender-Punkte entfernen
        txt = txt.replace(".", "")
        # Gedankenstriche/Leereinträge → 0
        if txt in ("–", "-", ""):
            txt = "0"
        row.append(txt)
    data.append(row)

In [22]:
cols = [
    "Bezirk",
    "Insgesamt",
    "PKW",
    "Omnibusse",
    "Lkw",
    "Zugmaschinen",
    "sonstige Kraftfahrzeuge",
    "Krafträder",
]
df = pd.DataFrame(data, columns=cols)

In [23]:
for col in cols[1:]:
    # unsichere Strings zu NaN, dann NaN → 0, schließlich int
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

In [24]:
print(df.info())
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Bezirk                   27 non-null     object
 1   Insgesamt                27 non-null     int64 
 2   PKW                      27 non-null     int64 
 3   Omnibusse                27 non-null     int64 
 4   Lkw                      27 non-null     int64 
 5   Zugmaschinen             27 non-null     int64 
 6   sonstige Kraftfahrzeuge  27 non-null     int64 
 7   Krafträder               27 non-null     int64 
dtypes: int64(7), object(1)
memory usage: 1.8+ KB
None
           Bezirk  Insgesamt     PKW  Omnibusse    Lkw  Zugmaschinen  \
0            Wien     924460  730947       4250  79340          4298   
1  1 Innere Stadt      19431   16160         22   1524            82   
2  2 Leopoldstadt      42711   34545          9   3453           136   
3    3 Landstraße      46